# 12.2.A Descriptive Statistics

We're going to dive deeper into working with data using `pandas`.  The libraries we'll use for this are `pandas` and `scipy`.

* load `pandas`
* load `stats` from `scipy`

In [ ]:
import pandas as pd
from scipy import stats

### Reading the data

Before we can dive into analyzing data, we need some data.  The connection string for the database we'll use is built for us already below.

* Use `pandas` to read the `ksprojects` table and assign it to a variable

In [ ]:
postgres_user = "dabc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "kickstarterprojects"

conn_str = f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"

In [ ]:
query = """
SELECT *
FROM ksprojects
"""

ks = pd.read_sql_query(query, conn_str)

### Inspecting the dataframe.

* How many rows are in the dataframe? How many columns?

* What are the columns' datatypes?  (use `.info()` here, but note that `.dtypes` is another option)

* How many unique values are there in the `'category'` column

* What `'main_category'` occurs the most in our dataset?

* What else might we explore?

### Missing Values

Note that when we used `.info()` we saw that the `'usd_pledged'` column had some missing values (i.e. its `non-null` count didn't match the row count).  Another way to check for missing values is by using `.isna()`.  This `.isna()` method is very commonly paired with `.sum()` (to get a count of the missing values per column) and `.mean()` (to get a percentage of the missing values per column).

* Call the `.isna()` method on our dataframe

* Pair `.isna()` with `.sum()` to get the counts of NAs per column

* Pair `.isna()` with `.mean()` to get the counts of NAs per column

### Dealing with missing values

So according to the above output, we've confirmed that the only column with NAs is the `'usd_pledged'` column.  How should we handle this?  In practice, the 2 main ways we might handle NAs are:

* Fill them in with a value
  * i.e. some people didn't mark their height on a survey, maybe we fill in with the average height reported

* Drop them


Before dropping we'd like to know why the values are NA because we might be dropping valid data.  For example, you might have seen something like below on a survey:

> 10. Are you a veteran?:  &emsp;&emsp; Yes [&emsp;]&emsp;&emsp;No [&emsp;]
> 
> (only answer the following questsions if you answered yes to question 10)

In this case, we should expect non-veterans to have missing values to the veteran only questions.  Other times, missing values aren't that deep and their just missing.  For example maybe a survey taker didn't know there was a page 2 so they left it all blank.

In practice it's very important to know how the data was collected and what a missing value indicates.

With all that said, I'm not sure the true reason for the missing values.  One way to explore this though is to use `.isna()` as a filter.  If you named your dataframe `ks`, then the below code will work to show all rows where `'usd_pledged'` is missing.

* What do we see these rows have in common besides the missing value?

In [ ]:
ks[ks["usd_pledged"].isna()]

#### Filling in NAs

To fill in missing values we can use the `.fillna()` method.  What would be a good value to fill in the missing `'usd_pledged'` column with? the average? 0? other? why?

* Use `.fillna()` with an appropriate value
* Save the result to a new variable
* Confirm that you no longer have missing values using `.isna()` and an aggregating function

#### Dropping NAs

In this case, we don't have a great explanation for why these values are missing so it's hard to say the best way to fill them in.  We also showed before that missing values account for less than 2% of the rows.  In practice, this is a case that dropping NAs might be the best option.

* Display the shape of the dataframe
* Use `.dropna()` to remove missing values (save the output to a variable)
* Display the shape of the resulting dataframe

### Descriptive statistics

One of the easiest ways we can get some descriptive stats about our dataframe's columns is with the `.describe()` method.

* Apply `.describe()` to our dataframe.
* What was the smallest `'goal'` that a kickstarter project asked for?
* Which column has a higher standard deviation, `'goal'` or `'pledged'`?

### Exploring relationships with statistics

Often times (esp in a business setting) we'd like to know which variables are related to each other.  Are sales higher in this season?  Do our ice cream sales go up when the temperature rises? etc

#### t-test revisited

One way to explore these relationships is our good old friend the t-test.  The t-test can let us declare if the average is significantly different between 2 groups.  (i.e. do we sell more ice cream in summer or winter? We have 2 groups, summer and winter, and we can compare the average sales per day between the groups).

Try and remember:

* What were the rules of thumb for when were allowed to perform a t-test? How do we know if our data is the right shape?
* What are the null and alternative hypotheses for a t-test?
* How do we know when the t-test is showing a statistically significant difference?

----

To perform a t-test in python we're going to be using the `scipy` package.   We can use this package to check the assumptions and to perform the actual test.

Before we can run a t-test, we need to split the data into 2 groups to compare.  Let's say our goal is to compare the `'Film & Video'` & `'Music'` main categories to see which has a higher amount of `'usd_pledged'`.

* Split the dataframe into the 2 appropriate groups

* Use the `stats.describe()` function on the `'usd_pledged'` column of each of the 2 groups
* Are we good to move forward with a t-test?  If yes, perform the t-test using `stats.ttest_ind()`.

----

* Read in the `people` table from the `baseball` database
* Drop any NAs

In [ ]:
postgres_db = "baseball"
conn_str = f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"

query = """
SELECT *
FROM people
"""

players = pd.read_sql_query(query, conn_str)
players = players.dropna()
players.head(2)

* Split the dataframe into 2 groups: those that bat right handed and those that bat left handed

In [ ]:
righties = players[players["bats"] == "R"]
lefties = players[players["bats"] == "L"]

* Use `stats.describe` to see if we can run a t-test to determine if the height is different between these 2 groups

In [ ]:
stats.describe(righties["height"])

In [ ]:
stats.describe(lefties["height"])

* Run a t-test using `stats.ttest_ind()`
* Is there a significant difference?

#### Correlation

A t-test is appropriate when we have a numeric column split into 2 groups by a categorical column.  If we want to explore relationships between 2 numeric columns we might use a correlation.  The correlation between 2 numeric columns ranges between `[-1, 1]`.

* A correlation of -1 is a strong negative correlation
  * For example, `amount of money spent` and `amount of money saved` would be negatively correlated.  As the `amount of money spent` goes up the `amount of money saved` would go down and vice versa.
* A correlation of 0 is a weak correlation
  * For example, the `number of words in the harry potter books` and the `number of arrests in costa rica` are likely not very related.
* A correlation of 1 is a strong positive correlation
  * For example, `amount of ice cream sold` and `temperature` are likely positively correlated.  As the `temperature` goes up the `amount of ice cream sold` likely goes up as well.
  
Note the classic phrase: "correlation does not equal causation".  Just because 2 things are related doesn't mean one thing causes the other ([this site](https://www.tylervigen.com/spurious-correlations) has examples of correlations that are 'spurious' that is, the 2 variables appear related, but they likely aren't).

To run a correlation on a `pandas` dataframe we can use `.corr()`.

* Apply `.corr()` to our dataframe
* What numeric variables are related? is this a positive or negative relationship?

Similiarly to t-test, we might want to declare if a correlation is 'significant' or not.  To do this in python we can use `stats.pearsonr()` and pass in our 2 variables like we did for a t-test.  The hypotheses for this test are:

* $H_0$: There is not a significant correlation
* $H_A$: There is a significant correlation

----

* Run `stats.pearsonr()` on the kickstarter dataframe to see if there's a significant correlation between the number of `'backers'` and the amount of `'usd_pledged'`
* Is the correlation significant?